# COSTO UNITARIO PROMEDIO

### OBTENCION COSTO DEL PRODUCTO

In [6]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# 1. Obtener las órdenes de producción ya cerradas (hechas por manufactura)
ordenes_produccion = models.execute_kw(
    db, uid, password,
    'mrp.production', 'search_read',
    [[('state', '=', 'done')]],   # Solo cerradas
    {'fields': ['id', 'product_id', 'product_qty']}
)

# 2. Extraer los productos únicos de esas órdenes
ids_productos_manu = list({op['product_id'][0] for op in ordenes_produccion if op.get('product_id')})

# 3. Leer los productos manufacturados que correspondan a DONSSON
productos_manu = models.execute_kw(
    db, uid, password,
    'product.product', 'read',
    [ids_productos_manu],
    {'fields': ['id', 'name', 'standard_price', 'catalogo_codigo_fab', 'qty_available']}
)

# 4. Filtrar solo marca DONSSON
productos_donsson_manu = [p for p in productos_manu if p.get('catalogo_codigo_fab') == 'DONSSON']



productos_manu = pd.DataFrame(productos_manu) # productos con costo entre los manufacturados


In [7]:
productos_manu = productos_manu.rename(columns={
    'name': 'producto',
    'standard_price': 'costo_estandar_promedio',
    'qty_available': 'cantidad_disponible'
})

pd.set_option("display.max_colwidth", None)
productos_manu = productos_manu[["producto","costo_estandar_promedio","cantidad_disponible"]]

#### Buscador para casos especiales productos

In [8]:
productos_manu[productos_manu["producto"].str.contains("DA2919", case=False, na=False)]

,producto,costo_estandar_promedio,cantidad_disponible
293,DA2919 FILTROAIRE CHEV. LV150,45119.8,110.0


### OBTENCION COSTO DE ORDEN DE PRODUCCION

In [9]:
# 1. Obtener las órdenes de producción ya cerradas (hechas por manufactura)
ordenes_produccion = models.execute_kw(
    db, uid, password,
    'mrp.production', 'search_read',
    [[('state', '=', 'done'),("date_finished" , ">" , "2022-12-31")]],   # Solo cerradas
    {'fields': ['id','name'#'date_start'
                , 'product_id', 'product_qty','valor_unitario_mrp','date_finished']}
)



ops = pd.DataFrame(ordenes_produccion) # todas las OP cerradas

In [10]:
ops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4592 entries, 0 to 4591
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   valor_unitario_mrp  4592 non-null   float64
 1   product_id          4592 non-null   object 
 2   date_finished       4592 non-null   object 
 3   product_qty         4592 non-null   float64
 4   id                  4592 non-null   int64  
 5   name                4592 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 215.4+ KB


In [11]:
ops.head()

,valor_unitario_mrp,product_id,date_finished,product_qty,id,name
0,36862.48,"[17260, [DAB02926025] DA2926 FILTRO AIRE DONSSON - INTERNATIONAL 4300 - 7400 (025 DA2926)]",2025-10-31 15:53:18,500.0,13670,MO13400
1,298582.00,"[50229, [DAB02013025] DA2013 FILTRO AIRE - KENWORTH - PETERBILT (025 DA2013)]",2025-10-28 15:57:08,1.0,13659,MO13389
2,37570.72,"[49566, [DAB08221025] DA8221 FILTRO AIRE - FAW EN DESARROLLO (025 DA8221)]",2025-10-31 15:46:26,30.0,13655,MO13385
3,25006.64,"[17226, [DAB12872025] DA2872A FILTRO AIRE- AGRALE (025 DA2872A)]",2025-10-31 14:52:53,200.0,13638,MO13368
4,46399.38,"[17306, [DAB02982025] DA2982 FILTRO AIRE DONSSON - IHC 7600i MOTOR CUMMINS (025 DA2982)]",2025-10-31 10:17:44,500.0,13637,MO13367


In [12]:
ops = ops.rename(columns={

    "name" :"nombre_op",
    "product_qty":"unidades_producidas",
    "valor_unitario_mrp":"costo_unitario_op",
    "date_finished":"fecha_cierre_op"
})


In [13]:
import re

# 1) Quedarnos con el nombre del producto dentro de product_id
#    (Odoo devuelve [id, "nombre"])
ops['product_name_raw'] = ops['product_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

# 2) Quitar el código entre corchetes "[...]" + espacios iniciales
# 3) Quitar todo lo que esté dentro de paréntesis "(...)" + espacios
ops['producto'] = (
    ops['product_name_raw']
      .astype(str)
      .str.replace(r'^\s*\[.*?\]\s*', '', regex=True)   # elimina "[CODE] "
      .str.replace(r'\(.*?\)', '', regex=True)          # elimina "(...)"
      .str.strip()
)

# (opcional) normalizar espacios internos
ops['producto'] = ops['producto'].str.replace(r'\s+', ' ', regex=True)




#### Buscador casos especiales por op

In [14]:
ops = ops[["producto","unidades_producidas","nombre_op","costo_unitario_op","fecha_cierre_op"]]
pd.set_option("display.max_colwidth", None)

ops[ops["producto"].str.contains("DA2983", case=False, na=False)]



,producto,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op
267,DA2983 FILTROAIRE MOTORES PERKINS,200.0,MO13059,10804.43,2025-09-12 13:51:18
884,DA2983 FILTROAIRE MOTORES PERKINS,250.0,MO12313,10023.63,2025-05-07 10:11:22
1081,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO12113,11036.00,2025-03-28 07:39:47
1482,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO11704,12236.66,2024-12-27 14:55:34
1855,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO11292,10042.45,2024-09-27 15:55:31
2273,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO10871,11038.63,2024-07-11 06:44:08
2744,DA2983 FILTROAIRE MOTORES PERKINS,200.0,MO10367,11866.00,2024-04-30 15:08:05
3315,DA2983 FILTROAIRE MOTORES PERKINS,300.0,MO09748,10816.48,2023-11-08 09:22:11
3760,DA2983 FILTROAIRE MOTORES PERKINS,300.0,MO09266,10458.89,2023-07-26 13:46:17
4115,DA2983 FILTROAIRE MOTORES PERKINS,151.0,MO08876,11377.29,2023-05-17 16:02:46


### Corrector matches merge

### OPS DE CADA PRODUCTO

In [15]:
from fuzzywuzzy import process

# 1. Merge exacto
df_unido = productos_manu.merge(ops, on="producto", how="left", indicator=True)

# 2. Productos que no hicieron match exacto
no_match = df_unido[df_unido["_merge"] == "left_only"]["producto"].unique()

# 3. Lista de productos en OP
productos_ops = ops["producto"].dropna().unique()

# 4. Crear diccionario de equivalencias con fuzzy
mapeo = {}
for prod in no_match:
    mejor_match, score = process.extractOne(prod, productos_ops)
    if score >= 85:   # solo aceptamos matches muy altos
        mapeo[prod] = mejor_match

# 5. Reemplazar en productos_manu
productos_manu["producto_corr"] = productos_manu["producto"].replace(mapeo)

# 6. Nuevo merge usando los nombres corregidos
df_unido_corr = productos_manu.merge(ops, left_on="producto_corr", right_on="producto", how="left", indicator=True)



/home/donsson/proyectos/env/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [16]:
#df_unido_corr[df_unido_corr["producto_x"].str.contains("DA2983")]

### COSTO DE CADA OP

In [17]:
from fuzzywuzzy import process

# ==============================
# 1) Calcular costo promedio real de productos_manu
# ==============================
def calcular_costo_promedio(grupo):
    if grupo["cantidad_disponible"].sum() == 0:
        return None  # evitar división por 0
    return (grupo["costo_estandar_promedio"] * grupo["cantidad_disponible"]).sum() / grupo["cantidad_disponible"].sum()

costo_promedio = (
    productos_manu.groupby("producto")
    .apply(calcular_costo_promedio)
    .reset_index(name="costo_promedio")
)

# ==============================
# 2) Merge exacto primero (ops vs productos_manu)
# ==============================
df_ops_unido = ops.merge(costo_promedio, on="producto", how="left", indicator=True)

# ==============================
# 3) Detectar OPs sin match
# ==============================
no_match_ops = df_ops_unido[df_ops_unido["_merge"] == "left_only"]["producto"].unique()

# ==============================
# 4) Lista de productos válidos (de productos_manu)
# ==============================
productos_manu_lista = costo_promedio["producto"].dropna().unique()

# ==============================
# 5) Crear diccionario de equivalencias fuzzy
# ==============================
mapeo_ops = {}
for prod in no_match_ops:
    mejor_match, score = process.extractOne(prod, productos_manu_lista)
    if score >= 85:
        mapeo_ops[prod] = mejor_match

# ==============================
# 6) Corregir los nombres en ops
# ==============================
ops["producto_corr"] = ops["producto"].replace(mapeo_ops)

# ==============================
# 7) Merge final (con fuzzy corregido)
# ==============================
ops_con_costo = ops.merge(
    costo_promedio, left_on="producto_corr", right_on="producto", how="left"
)

# ==============================
# 8) Limpiar columnas duplicadas
# ==============================

cols_a_borrar = [c for c in ["producto_x", "producto_y"] if c in ops_con_costo.columns]
ops_con_costo = ops_con_costo.drop(columns=cols_a_borrar)

# Renombrar la corregida como "producto"
if "producto_corr" in ops_con_costo.columns:
    ops_con_costo = ops_con_costo.rename(columns={"producto_corr": "producto"})


#Tarda 2 min aprox


/tmp/ipykernel_12077/471142342.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calcular_costo_promedio)


In [18]:
ops_con_costo[ops_con_costo["producto"].str.contains("DA2570")]

,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op,producto,costo_promedio
43,1000.0,MO13301,14082.55,2025-10-15 16:25:56,"DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.",13771.02
153,1000.0,MO13177,13975.37,2025-09-19 14:48:48,"DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.",13771.02
282,1000.0,MO13044,13678.95,2025-08-28 13:24:48,"DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.",13771.02
366,1001.0,MO12917,13876.29,2025-08-14 16:13:39,"DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.",13771.02
456,350.0,MO12779,18539.43,2025-07-21 09:59:32,DA2570UHE FILTRO DE AIRE ALTA EFICIENCIA - CAT - KUBOTA - CASE,18775.78
486,1003.0,MO12749,12931.15,2025-07-21 09:58:51,"DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.",13771.02
642,1000.0,MO12593,13547.66,2025-06-24 09:17:15,"DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.",13771.02
726,1200.0,MO12507,13504.28,2025-05-31 11:39:40,"DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.",13771.02
783,200.0,MO12415,19908.99,2025-05-23 10:52:57,DA2570UHE FILTRO DE AIRE ALTA EFICIENCIA - CAT - KUBOTA - CASE,18775.78
834,1100.0,MO12364,14750.02,2025-05-19 14:26:17,"DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.",13771.02


In [19]:
ops_con_costo['diferencia'] = ops_con_costo['costo_unitario_op'] - ops_con_costo['costo_promedio']

ops_con_costo["%diferencia"] = (ops_con_costo["diferencia"] / ops_con_costo["costo_promedio"])

ops_con_costo["%_diferencia_abs"] = (ops_con_costo["diferencia"].abs() / ops_con_costo['costo_promedio'])


In [20]:
ops_con_costo.to_excel("ops_relacionadas.xlsx")

In [21]:
ops_con_costo[ops_con_costo["costo_promedio"]==0]  # Comprobamos que no quedo ninguna OP sin el costo promedio asignado

,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op,producto,costo_promedio,diferencia,%diferencia,%_diferencia_abs


### Df unido y corregido

In [22]:
df_unido_corr[df_unido_corr["producto_x"].str.contains("DA2983")].sample(10)

,producto_x,costo_estandar_promedio,cantidad_disponible,producto_corr,producto_y,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op,_merge
2558,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,248.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,250.0,MO12313,10023.63,2025-05-07 10:11:22,both
2565,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,248.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,300.0,MO09266,10458.89,2023-07-26 13:46:17,both
2557,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,248.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,200.0,MO13059,10804.43,2025-09-12 13:51:18,both
2564,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,248.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,300.0,MO09748,10816.48,2023-11-08 09:22:11,both
2563,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,248.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,200.0,MO10367,11866.00,2024-04-30 15:08:05,both
2560,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,248.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO11704,12236.66,2024-12-27 14:55:34,both
2559,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,248.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO12113,11036.00,2025-03-28 07:39:47,both
2561,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,248.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO11292,10042.45,2024-09-27 15:55:31,both
2562,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,248.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,150.0,MO10871,11038.63,2024-07-11 06:44:08,both
2567,DA2983 FILTROAIRE MOTORES PERKINS,10686.95,248.0,DA2983 FILTROAIRE MOTORES PERKINS,DA2983 FILTROAIRE MOTORES PERKINS,200.0,MO08426,13402.83,2023-02-04 11:57:28,both


In [23]:
# 1. Crear columna final de producto
df_unido_corr["producto"] = df_unido_corr["producto_corr"].fillna(df_unido_corr["producto_x"])

# 2. Eliminar columnas duplicadas
df_unido_corr = df_unido_corr.drop(columns=["producto_x", "producto_y", "producto_corr"], errors="ignore")

# 3. Eliminar duplicados de OP
df_unido_corr = df_unido_corr.drop_duplicates(subset=["nombre_op", "fecha_cierre_op", "producto"], keep="last")

# 4. Ordenar
df_limpio = df_unido_corr.sort_values(["producto", "fecha_cierre_op"], ascending=[True, False]).reset_index(drop=True)


In [24]:
df_limpio = df_limpio[~ (df_limpio['cantidad_disponible']==0)]  #Retirar ruido , productos sin unidades disponibles


df_limpio[df_limpio["producto"].str.contains("DA2983")]

,costo_estandar_promedio,cantidad_disponible,unidades_producidas,nombre_op,costo_unitario_op,fecha_cierre_op,_merge,producto
1742,10686.95,248.0,200.0,MO13059,10804.43,2025-09-12 13:51:18,both,DA2983 FILTROAIRE MOTORES PERKINS
1743,10686.95,248.0,250.0,MO12313,10023.63,2025-05-07 10:11:22,both,DA2983 FILTROAIRE MOTORES PERKINS
1744,10686.95,248.0,150.0,MO12113,11036.00,2025-03-28 07:39:47,both,DA2983 FILTROAIRE MOTORES PERKINS
1745,10686.95,248.0,150.0,MO11704,12236.66,2024-12-27 14:55:34,both,DA2983 FILTROAIRE MOTORES PERKINS
1746,10686.95,248.0,150.0,MO11292,10042.45,2024-09-27 15:55:31,both,DA2983 FILTROAIRE MOTORES PERKINS
1747,10686.95,248.0,150.0,MO10871,11038.63,2024-07-11 06:44:08,both,DA2983 FILTROAIRE MOTORES PERKINS
1748,10686.95,248.0,200.0,MO10367,11866.00,2024-04-30 15:08:05,both,DA2983 FILTROAIRE MOTORES PERKINS
1749,10686.95,248.0,300.0,MO09748,10816.48,2023-11-08 09:22:11,both,DA2983 FILTROAIRE MOTORES PERKINS
1750,10686.95,248.0,300.0,MO09266,10458.89,2023-07-26 13:46:17,both,DA2983 FILTROAIRE MOTORES PERKINS
1751,10686.95,248.0,151.0,MO08876,11377.29,2023-05-17 16:02:46,both,DA2983 FILTROAIRE MOTORES PERKINS


In [25]:
# ===========================
# 1) Función costo real
# ===========================
def calcular_costo_real(grupo):
    # ordenar las OP por fecha (descendente: más recientes primero)
    grupo = grupo.sort_values('fecha_cierre_op', ascending=False)

    cant_disp = grupo['cantidad_disponible'].iloc[0]
    total_tomado = 0
    costo_total = 0

    for _, row in grupo.iterrows():
        if total_tomado >= cant_disp:
            break

        # ignorar OP con costo nulo o cero
        if pd.isna(row['costo_unitario_op']) or row['costo_unitario_op'] == 0:
            continue

        # Tomar lo que falta o lo que tiene esta OP
        tomar = min(row['unidades_producidas'], cant_disp - total_tomado)

        costo_total += tomar * row['costo_unitario_op']
        total_tomado += tomar

    if total_tomado == 0:
        return None

    return costo_total / total_tomado


# ===========================
# 2) Costo real por producto
# ===========================
costo_real = (
    df_limpio
    .groupby('producto')
    .apply(calcular_costo_real)
    .reset_index()
)
costo_real.columns = ['producto', 'costo_real']


# ===========================
# 3) Última fecha de producción por producto
# ===========================
ultima_fecha = (
    df_limpio.groupby('producto')['fecha_cierre_op']
    .max()
    .reset_index()
)
ultima_fecha.columns = ['producto', 'ultima_fecha_produccion']


# ===========================
# 4) Construcción resumen
# ===========================
resumen = (
    df_limpio[['producto', 'costo_estandar_promedio', 'cantidad_disponible']]
    .drop_duplicates()
    .merge(costo_real, on='producto', how='left')
    .merge(ultima_fecha, on='producto', how='left')   # <-- se agrega la fecha
)

# Diferencia entre costo real y costo estándar
resumen['diferencia'] = resumen['costo_real'] - resumen['costo_estandar_promedio']

resumen["%diferencia"] = (resumen["diferencia"] / resumen["costo_estandar_promedio"])

resumen["%_diferencia_abs"] = (resumen["diferencia"].abs() / resumen["costo_estandar_promedio"])


/tmp/ipykernel_12077/3487751595.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(calcular_costo_real)


In [26]:
resumen[resumen["costo_real"].isna()] # es un producto que tiene existencias y tiene una op pero en ella no cargaron costo unitario ejecutado fue en 2017

,producto,costo_estandar_promedio,cantidad_disponible,costo_real,ultima_fecha_produccion,diferencia,%diferencia,%_diferencia_abs


In [39]:
resumen[resumen["producto"].str.contains("DA4982", case=False, na=False)]


,producto,costo_estandar_promedio,cantidad_disponible,costo_real,ultima_fecha_produccion,diferencia,%diferencia,%_diferencia_abs
369,DA4982 FILTRO AIRE INT. CAMION IHC 7600 MOTOR CUMMINS,24338.22,777.0,24481.565946,2025-09-30 16:25:24,143.345946,0.00589,0.00589


In [ ]:
resumen.to_excel("monitoreo_.xlsx")

In [29]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# 1. Obtener las órdenes de producción ya cerradas (hechas por manufactura)
precio_estandar_historico = models.execute_kw(
    db, uid, password,
    'stock.change.standard.price', 'search_read',
    [[]],   # Todo
    {'fields': ['create_date', 'product_id', 'product_qty']}
)



# EQUIVALENCIAS MARCAS

In [30]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

campos = models.execute_kw(
    db, uid, password,
    'product.template', 'fields_get',
    []
)

for nombre, info in campos.items():
    if info.get('type') in ('one2many', 'many2one', 'many2many'):
        print(nombre, "->", info.get('relation'))




write_uid -> res.users
property_account_income -> account.account
website_style_ids -> product.style
product_rate -> website.product.rate
property_stock_account_output -> account.account
catalogo_valvula_alivio -> catalogo.product.valvula_alivio
company_id -> res.company
public_categ_ids -> product.public.category
pricelist_id -> product.pricelist
property_stock_account_input -> account.account
bom_ids -> mrp.bom
classprod_id -> cs.clasprod
catalogo_material_filtrante -> catalogo.product.material_filtrante
alternative_product_ids -> product.template
seller_id -> res.partner
packaging_ids -> product.packaging
pos_categ_id -> pos.category
seller_ids -> product.supplierinfo
website_message_ids -> mail.message
catalogo_marca -> catalogo.marcas
accessory_product_ids -> product.product
message_follower_ids -> res.partner
attribute_line_ids -> product.attribute.line
asset_category_id -> account.asset.category
images -> product.image
catalogo_rosca -> catalogo.product.rosca
catalogo_forma -> c

In [31]:
import xmlrpc.client
import pandas as pd

username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


# 1) Productos básicos
productos = models.execute_kw(
    db, uid, password,
    'product.template', 'search_read',
    [[]],
    {'fields': ['id', 'name', 'default_code']}
)
df_prod = pd.DataFrame(productos).rename(columns={
    'id': 'product_id',
    'name': 'producto_nombre',
    'default_code': 'producto_codigo'
})

# 2) Equivalencias
equivalencias = models.execute_kw(
    db, uid, password,
    'catalogo.referencia.fabricantes', 'search_read',
    [[]],
    {'fields': ['id', 'name', 'tipo_marca', 'productos_referencia']}
)
df_eq = pd.DataFrame(equivalencias)

# 3) Expandir la lista de productos_referencia
df_eq = df_eq.explode('productos_referencia')
df_eq = df_eq.rename(columns={'name': 'equivalencia_ref'})

# 4) Limpiar tipo_marca (de lista [id, nombre] → solo nombre)
df_eq['marca'] = df_eq['tipo_marca'].apply(lambda x: x[1] if isinstance(x, list) else None)
df_eq['productos_referencia'] = df_eq['productos_referencia'].astype('Int64')

# 5) Merge con productos
df_final = df_eq.merge(df_prod, left_on='productos_referencia', right_on='product_id', how='left')
df_final = df_final.dropna()

# 6) Exportar a Excel
#df_final[['producto_codigo', 'producto_nombre', 'equivalencia_ref', 'marca']].to_excel("equivalencias.xlsx", index=False)
print("✅ Archivo 'equivalencias.xlsx' generado correctamente")


✅ Archivo 'equivalencias.xlsx' generado correctamente


In [32]:
df_final["ref_donsson"] = df_final["producto_nombre"].str.split(' ').str[0]

In [33]:
df_final = df_final[['ref_donsson', 'producto_nombre','producto_codigo', 'equivalencia_ref', 'marca']].sort_values(by=["ref_donsson"])

df_final[df_final.duplicated()].head(10)

,ref_donsson,producto_nombre,producto_codigo,equivalencia_ref,marca
24997,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,42378,WIX
24994,DA2049,DA2049 FILTRO AIRE 1_CATERPILLAR,BAR02049125,42334,WIX
24993,DA2049,DA2049 FILTRO AIRE 1_ CATERPILLAR,DAR02049025,42334,WIX
65211,DA2110,"DA2110 FILTRO AIRE 1_ GMC,A. CHALMER,CLARK.",BAE02110125,P182041,DONALDSON
65210,DA2110,"DA2110 FILTRO AIRE 1_ GMC,A. CHALMER,CLARK.",DAR02110025,P182041,DONALDSON
24989,DA2110,"DA2110 FILTRO AIRE 1_ GMC,A. CHALMER,CLARK.",DAR02110025,42225,WIX
25040,DA2124,DA2124 FILTRO AIRE 1_ CATERPILLAR,DAR02124025,42980,WIX
25041,DA2124,DA2124 FILTRO AIRE 1_ CATERPILLAR,BAR02124125,42980,WIX
27152,DA2239,DA2239 FILTRO AIRE BRIGADIER CAREPANELA.,BAE02239125,57MD36M,MACK
66425,DA2242,DA2242 FILTRO AIRE 1_ KOMATSU,DAR02242025,42377,WIX


In [34]:
df_final[df_final["producto_codigo"]=="DAR02042025"].sort_values(by=["marca"])

,ref_donsson,producto_nombre,producto_codigo,equivalencia_ref,marca
19217,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,AZ216,A Y Z
35798,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,PA1673,BALDWIN
45373,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,2S1288,CATERPILLAR
4139,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,1W0392,CATERPILLAR
4152,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,1W392,CATERPILLAR
6293,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,2S1636,CATERPILLAR
9685,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,4M2976,CATERPILLAR
9687,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,4M3976,CATERPILLAR
12916,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,7W3920,CATERPILLAR
9698,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,4M9378,CATERPILLAR


In [35]:
df_def = df_final.drop_duplicates()

In [36]:
df_def[df_def["producto_codigo"]=="DAR02042025"].sort_values(by=["marca"])

,ref_donsson,producto_nombre,producto_codigo,equivalencia_ref,marca
19217,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,AZ216,A Y Z
35798,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,PA1673,BALDWIN
45373,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,2S1288,CATERPILLAR
4139,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,1W0392,CATERPILLAR
4152,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,1W392,CATERPILLAR
6293,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,2S1636,CATERPILLAR
9685,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,4M2976,CATERPILLAR
9687,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,4M3976,CATERPILLAR
12916,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,7W3920,CATERPILLAR
9698,DA2042,DA2042 FILTRO AIRE 1_ CATERPILLAR,DAR02042025,4M9378,CATERPILLAR


In [37]:
df_def.to_excel("equivalencias.xlsx", index=False)
df_def.head(20)

,ref_donsson,producto_nombre,producto_codigo,equivalencia_ref,marca
54937,DA2001,DA2001 FILTRO AIRE 1o. CAMIONES VOLVO,BAB02001125,RS3740,BALDWIN
54938,DA2001,DA2001 FILTRO AIRE 1o. CAMIONES VOLVO,BAB02001125,8076195,VOLVO
54939,DA2001,DA2001 FILTRO AIRE 1o. CAMIONES VOLVO,BAB02001125,P540388,DONALDSON
54940,DA2001,DA2001 FILTRO AIRE 1o. CAMIONES VOLVO,BAB02001125,AF25435,FLEETGUARD
55936,DA2001,DA2001 FILTRO AIRE 1o. CAMIONES VOLVO,BAB02001125,8082103,VOLVO
47513,DA2002,DA2002 FILTRO AIRE CAMIONES OTTAWA MOTOR CUMMINS,BAE02002125,PA3493,BALDWIN
47887,DA2002,DA2002 FILTRO AIRE CAMIONES OTTAWA MOTOR CUMMINS,BAE02002125,LAF1934,LUBERFINER
47886,DA2002,DA2002 FILTRO AIRE CAMIONES OTTAWA MOTOR CUMMINS,BAE02002125,P537454,DONALDSON
47885,DA2002,DA2002 FILTRO AIRE CAMIONES OTTAWA MOTOR CUMMINS,BAE02002125,CA8130,FRAM
47884,DA2002,DA2002 FILTRO AIRE CAMIONES OTTAWA MOTOR CUMMINS,BAE02002125,AH1197,FLEETGUARD


In [38]:
df_def.isna().sum()

ref_donsson         0
producto_nombre     0
producto_codigo     0
equivalencia_ref    0
marca               0
dtype: int64